In [ ]:
!pip install -q auto-gptq transformers accelerate optimum

In [ ]:
!pip install optimum

In [ ]:
!pip install auto-gptq

In [ ]:
# git 연동
!git clone https://github.com/CampusChatBot-Team5/Termproject_5.git

In [ ]:
# 패키지 임포트
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from google.colab import files
import pandas as pd
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [ ]:
# 모델 로드 8B -> 4bit 양자화 LLama 기반 (한국어 모델)
model_name = "allganize/Llama-3-Alpha-Ko-8B-Instruct-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

model = prepare_model_for_kbit_training(model)
lora_cfg = LoraConfig(r=8, lora_alpha=32, target_modules=["q_proj","v_proj"], bias="none")
model = get_peft_model(model, lora_cfg)

In [ ]:
#LLM -> 학교 공지, 학사 일정 관련 분류 모호하게함
# 시스템 프롬프트
system_prompt = """\
너는 충남대학교 관련 질문을 아래 다섯 개 카테고리 중 하나로 분류하는 분류기야.
모든 질문에 대해 반드시 해당되는 **숫자 하나만** 출력해야 해.

[카테고리 목록]
0: 졸업요건 – 졸업, 학점 이수, 인증, 졸업시험
1: 학교공지 – 장학금, 공모전, 특강, 예비군, 등록금, 행사
2: 학사일정 – 개강, 종강, 수강신청, 정정기간, 방학, 성적
3: 식단안내 – 오늘 식단, 학식 메뉴, 점심, 저녁, 건강식
4: 통학버스 – 버스 시간, 노선, 정류장, 운행

절대로 설명하지 말고, 반드시 숫자 하나만 출력해.
"""

# few-shot
fewshot_examples = [
    {"role": "user", "content": "오늘 점심 뭐 나와?"}, {"role": "assistant", "content": "3"},
    {"role": "user", "content": "예비군 훈련 어떻게 신청해?"}, {"role": "assistant", "content": "1"},
    {"role": "user", "content": "졸업하려면 몇 학점 들어야 해?"}, {"role": "assistant", "content": "0"},
    {"role": "user", "content": "통학버스 시간표 좀 알려줘"}, {"role": "assistant", "content": "4"},
    {"role": "user", "content": "수강신청 기간이 언제야?"}, {"role": "assistant", "content": "2"},
]

# system에 프롬프트와 few_shot 넣어주는 방식
def build_messages_with_fewshot(question: str):
    messages = [{"role": "system", "content": system_prompt}]
    messages += fewshot_examples
    messages.append({"role": "user", "content": question})
    return messages


In [ ]:
def generate_answer(question, max_new_tokens=5):
    messages = build_messages_with_fewshot(question)

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.05,
    )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)


def classify_question(question, retries=3):
    for _ in range(retries):
        output = generate_answer(question, max_new_tokens=5)
        try:
            result = output.strip().split()[0]
        except:
            continue
        if result in ['0', '1', '2', '3', '4']:
            return result
    return "예측 실패"

In [ ]:
def make_prompt(question):
    return question  # 직접 사용 안함 (ChatML 사용 중)

def keyword_or_llm_classify(question, alpha_if_keyword=0.9):
    q = question.lower()
    keyword_map = {
        0: ["졸업", "이수", "학점", "인증", "시험"],
        1: ["장학금", "공지", "예비군", "등록금", "공모전", "특강"],
        2: ["개강", "종강", "방학", "수강", "성적", "정정", "학사", "휴강"],
        3: ["학식", "메뉴", "점심", "저녁", "식당", "다이어트", "건강", "식단"],
        4: ["버스", "통학", "정류장", "노선", "운행"]
    }

    keyword_scores = [0] * 5
    for label, keywords in keyword_map.items():
        keyword_scores[label] = sum(1 for kw in keywords if kw in q)

    total_kw = sum(keyword_scores)
    keyword_probs = [s / total_kw for s in keyword_scores] if total_kw > 0 else [0] * 5
    alpha = alpha_if_keyword if total_kw > 0 else 0.0

    result = classify_question(question)
    llm_probs = [0] * 5
    if result in ['0', '1', '2', '3', '4']:
        llm_probs[int(result)] = 1

    final_scores = [alpha * kw + (1 - alpha) * llm for kw, llm in zip(keyword_probs, llm_probs)]

    print("question:", question)
    print("keyword_scores:", keyword_scores)
    print("LLM result:", result)
    print("final_scores:", final_scores)

    return str(final_scores.index(max(final_scores)))


In [ ]:
question = "계절학기 언제임?"
predicted_label = keyword_or_llm_classify(question)
print("최종 예측 라벨:", predicted_label)

In [ ]:
# 데이터 업로드
file_path = "/content/drive/MyDrive/data/test_cls_edit.json"

with open(file_path, "r", encoding="utf-8") as f:
    dataset = json.load(f)

In [ ]:
dataset[:5]

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import pandas as pd

y_true = []
y_pred = []
misclassified = []
fail_count = 0  # 예측 실패 개수

for sample in dataset:
    true_label = str(sample["label"])
    question = sample["question"]
    pred_label = keyword_or_llm_classify(question)

    if pred_label == "예측 실패":
        fail_count += 1
        continue

    y_true.append(true_label)
    y_pred.append(pred_label)

    if true_label != pred_label:
        misclassified.append((question, true_label, pred_label))

# 전체 성능 지표
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
recall = recall_score(y_true, y_pred, average="macro", zero_division=0)
f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)

# 라벨별 상세 지표
report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.loc[['0', '1', '2', '3', '4']]

# 정확도 계산
correct_counts = {str(i): 0 for i in range(5)}
total_counts = {str(i): 0 for i in range(5)}
for yt, yp in zip(y_true, y_pred):
    total_counts[yt] += 1
    if yt == yp:
        correct_counts[yt] += 1
df_report["accuracy"] = [
    correct_counts[label] / total_counts[label] if total_counts[label] else 0.0
    for label in df_report.index
]

# 출력
print("\n📊 전체 평가 지표")
print(f"- Accuracy:  {accuracy:.4f}")
print(f"- Precision: {precision:.4f}")
print(f"- Recall:    {recall:.4f}")
print(f"- F1 Score:  {f1:.4f}")

print("\n📌 라벨별 성능 지표")
print(df_report[["precision", "recall", "f1-score", "accuracy"]])

print(f"\n❌ 잘못 분류된 샘플 수: {len(misclassified)}개")
for i, (question, true_label, pred_label) in enumerate(misclassified[:10]):
    print(f"{i+1}. Q: {question} → 실제: {true_label}, 예측: {pred_label}")

print(f"\n⚠️ 예측 실패한 샘플 수: {fail_count}개")


In [ ]:
#졸업” 키워드 과적합 → label 0 과잉 분류
#“언제 떴나요?”, “공지 올라왔어요?” → 대부분 label 1 (공지성) → LLM 프롬프트 보강 필요
# 성적 정정 가능한 기간 공지됐나요? label 1 , label 2 어떤거??

## 프람프트 튜닝
label만 출력하도록 프람프트를 설계, 모델이 label을 출력하지 않을 경우 계속 요청
JSON 형식 강제
중립 답변 방지: "There is no neutral answer" 명시
부정확한 응답 대비: "neutral", "mixed" 같은 잘못된 답변 방지 문구 삽입
재시도 루프 구성: 잘못된 응답이 나올 경우, 반복해서 재요청함

→ 모델 응답 실패율을 낮추고, 일관성 있는 응답을 유도

## voting 방식
단일 LLM 결과 대신 여러 LLM의 예측 결과를 종합해 최종 판단.
적용된 방법:
Hard Voting: 5개 모델 중 가장 많이 나온 라벨을 선택
Soft Voting: 모델들의 확률 기반 응답 평균으로 가장 확률 높은 라벨 선택
→ 특히 성능 낮은 모델들의 soft voting 결과가 단독 모델보다 더 나은 성능을 보임

## 키워드 기반
label에 해당하는 키워드가 있으면 그 label로 분류함.

-> 키워드 분류 답 + LLM이 분류한 답

오늘 2학메뉴 뭐야? -> 메뉴 키워드 도출이 안됨

형태소 분석기로 분류를 해줘야 하나??